In [1]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install line-bot-sdk flask flask-ngrok

     |████████████████████████████████| 87 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 25.7 MB/s 
     |████████████████████████████████| 271 kB 61.8 MB/s 
     |████████████████████████████████| 144 kB 68.3 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 


In [3]:
!pip install pyngrok
!pyngrok authtoken 29JlwqUGLZxPpC0QfilJEO8AwsW_xxCAFaGtukqQ8krP2CFd

     |████████████████████████████████| 745 kB 4.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=0628cd71acec05b656dedd6649e852fd79921df314c19e197192d2e04101a0ac
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [5]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("LINE_CHANNEL_ACCESS_TOKEN")
handler = WebhookHandler("LINE_CHANNEL_SECRET")

In [6]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [7]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="text2"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="Data1")
)

In [8]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

In [9]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [10]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@reply":quick_reply_text_send_message,
}

In [11]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from linebot.models import PostbackEvent

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，我們會派專家您聯繫'
                )
            )
    else:
        pass

In [15]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@reply")
        )

In [16]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://aa97-104-199-162-99.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/May/2022 07:07:23] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[]}


...

{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106827809036","text":"@reply"},"webhookEventId":"01G3AZZ0KP0AE09W4719HMH2KK","deliveryContext":{"isRedelivery":false},"timestamp":1652857733529,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"d4b4ecdd9c0d44d39fb476aa2ebb1898","mode":"active"}]}
{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"postback","postback":{"data":"Data1"},"webhookEventId":"01G3AZZ5XPY7K1Z2FPYNCTNN3M","deliveryContext":{"isRedelivery":false},"timestamp":1652857738681,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"af33e27a41ed4561ab8987da89ffe36a","mode":"active"}]}


127.0.0.1 - - [18/May/2022 07:09:00] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 07:09:06] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106828762497","text":"@reply"},"webhookEventId":"01G3AZZC8PP7510BFTMMC6JFS6","deliveryContext":{"isRedelivery":false},"timestamp":1652857745176,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"0320579e10ed4f588ba53ce6f8dfbaf3","mode":"active"}]}


127.0.0.1 - - [18/May/2022 07:09:11] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106829172540","text":"text2"},"webhookEventId":"01G3AZZGZKEGBQ27YA7KW6T2HW","deliveryContext":{"isRedelivery":false},"timestamp":1652857750070,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"1feeec7f36344687af32e0785ac9a6f4","mode":"active"}]}
